In [1]:
!pip install pandas openpyxl transformers torch
!pip install llama-cpp-python


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from llama_cpp import Llama
from concurrent.futures import ThreadPoolExecutor

model_path = "/Users/briannapatten/Desktop/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
llm = Llama(model_path=model_path)

llama_model_load_from_file_impl: using device Metal (Apple M2 Pro) - 10922 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/briannapatten/Desktop/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:          

In [15]:
import random

def translate_query(query):
    #prompt = f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Return all occurrences of'.\n\n{query}\n\n### Response:"
    #Changes up prompts each time to ensure variety in English conversions
    PROMPTS = [
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Return all occurrences of'.\n\n{query}\n\n### Response:",
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Find where'.\n\n{query}\n\n### Response:",
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Return issuers that'.\n\n{query}\n\n### Response:",
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence.\n\n{query}\n\n### Response:",
        f"### Instruction:\nRephrase the following ESGish query into a natural English sentence that begins with 'Return'.\n\n{query}\n\n### Response:"
    ]
    #Error handling 
    try:
        #Chooses a random prompt from the array, queries a response from Mistral, returns response
        prompt = random.choice(PROMPTS).format(query=query)
        response = llm(prompt, max_tokens=575, temperature=0.1)
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(f"Error translating query: '{query}'\nError: {str(e)}")
        return "[TRANSLATION_FAILED]"

In [ ]:
import pandas as pd
from functools import partial
import time

#Reads and stores the Esgish queries
df = pd.read_excel("esgish.xlsx")
queries = df["Esgish"].tolist()
#Ensures no overload and efficiency
batch_size = 100 
output_file = "translated_queries.xlsx"

#Looks at each query in each batch, calls the translate_query function, and stores it
for i in range(0, len(queries), batch_size):
    batch = queries[i:i + batch_size]
    translated_batch = []
    
    for query in batch:
        translated = translate_query(query)
        translated_batch.append(translated)
    
    df_batch = pd.DataFrame({
        "Esgish": batch,
        "English": translated_batch
    })

    #Makes a new file if needed, or adds onto the current file during each batch in case the program crashes at some point
    if i == 0:
        df_batch.to_excel(output_file, index=False)  
    else:
        with pd.ExcelWriter(output_file, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
            df_batch.to_excel(writer, index=False, header=False, startrow=i + 1)
    
    print(f"Saved batch {i // batch_size + 1} to {output_file}")
    #Avoids overwhelming the system/API
    time.sleep(10) 

Llama.generate: 25 prefix-match hit, remaining 69 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =    1870.76 ms /    69 tokens (   27.11 ms per token,    36.88 tokens per second)
llama_perf_context_print:        eval time =    3010.59 ms /    62 runs   (   48.56 ms per token,    20.59 tokens per second)
llama_perf_context_print:       total time =    4890.39 ms /   131 tokens
Llama.generate: 42 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =     551.78 ms /    13 tokens (   42.44 ms per token,    23.56 tokens per second)
llama_perf_context_print:        eval time =    1303.88 ms /    27 runs   (   48.29 ms per token,    20.71 tokens per second)
llama_perf_context_print:       total time =    1859.27 ms /    40 tokens
Llama.generate: 39 prefix-match hit, remaining 9 prompt tokens to eval
llama_perf_cont

Saved batch 1 to translated_queries.xlsx


Llama.generate: 24 prefix-match hit, remaining 22 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =     654.94 ms /    22 tokens (   29.77 ms per token,    33.59 tokens per second)
llama_perf_context_print:        eval time =     642.73 ms /    13 runs   (   49.44 ms per token,    20.23 tokens per second)
llama_perf_context_print:       total time =    1300.69 ms /    35 tokens
Llama.generate: 20 prefix-match hit, remaining 20 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =     623.56 ms /    20 tokens (   31.18 ms per token,    32.07 tokens per second)
llama_perf_context_print:        eval time =     341.18 ms /     7 runs   (   48.74 ms per token,    20.52 tokens per second)
llama_perf_context_print:       total time =     966.07 ms /    27 tokens
Llama.generate: 20 prefix-match hit, remaining 25 prompt tokens to eval
llama_perf_con

Saved batch 2 to translated_queries.xlsx


Llama.generate: 54 prefix-match hit, remaining 28 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =    1022.25 ms /    28 tokens (   36.51 ms per token,    27.39 tokens per second)
llama_perf_context_print:        eval time =    1769.04 ms /    34 runs   (   52.03 ms per token,    19.22 tokens per second)
llama_perf_context_print:       total time =    2796.90 ms /    62 tokens
Llama.generate: 25 prefix-match hit, remaining 42 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =    1707.86 ms /    42 tokens (   40.66 ms per token,    24.59 tokens per second)
llama_perf_context_print:        eval time =     970.32 ms /    20 runs   (   48.52 ms per token,    20.61 tokens per second)
llama_perf_context_print:       total time =    2681.30 ms /    62 tokens
Llama.generate: 24 prefix-match hit, remaining 27 prompt tokens to eval
llama_perf_con

...

Llama.generate: 43 prefix-match hit, remaining 68 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =    1834.74 ms /    68 tokens (   26.98 ms per token,    37.06 tokens per second)
llama_perf_context_print:        eval time =    3208.74 ms /    66 runs   (   48.62 ms per token,    20.57 tokens per second)
llama_perf_context_print:       total time =    5054.84 ms /   134 tokens
Llama.generate: 25 prefix-match hit, remaining 103 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =    2073.88 ms /   103 tokens (   20.13 ms per token,    49.67 tokens per second)
llama_perf_context_print:        eval time =    2984.48 ms /    61 runs   (   48.93 ms per token,    20.44 tokens per second)
llama_perf_context_print:       total time =    5067.70 ms /   164 tokens
Llama.generate: 24 prefix-match hit, remaining 112 prompt tokens to eval
llama_perf_c

Saved batch 32 to translated_queries.xlsx


Llama.generate: 20 prefix-match hit, remaining 64 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =    1690.97 ms /    64 tokens (   26.42 ms per token,    37.85 tokens per second)
llama_perf_context_print:        eval time =    2484.03 ms /    51 runs   (   48.71 ms per token,    20.53 tokens per second)
llama_perf_context_print:       total time =    4183.55 ms /   115 tokens
Llama.generate: 20 prefix-match hit, remaining 75 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =    1822.86 ms /    75 tokens (   24.30 ms per token,    41.14 tokens per second)
llama_perf_context_print:        eval time =    2333.77 ms /    48 runs   (   48.62 ms per token,    20.57 tokens per second)
llama_perf_context_print:       total time =    4163.93 ms /   123 tokens
Llama.generate: 25 prefix-match hit, remaining 96 prompt tokens to eval
llama_perf_con

Saved batch 33 to translated_queries.xlsx


Llama.generate: 25 prefix-match hit, remaining 135 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =    2473.06 ms /   135 tokens (   18.32 ms per token,    54.59 tokens per second)
llama_perf_context_print:        eval time =   35849.90 ms /   100 runs   (  358.50 ms per token,     2.79 tokens per second)
llama_perf_context_print:       total time =   38377.59 ms /   235 tokens
Llama.generate: 20 prefix-match hit, remaining 131 prompt tokens to eval
llama_perf_context_print:        load time =    2109.69 ms
llama_perf_context_print: prompt eval time =   16692.45 ms /   131 tokens (  127.42 ms per token,     7.85 tokens per second)
llama_perf_context_print:        eval time =   25809.19 ms /    87 runs   (  296.66 ms per token,     3.37 tokens per second)
llama_perf_context_print:       total time =   42550.73 ms /   218 tokens
Llama.generate: 36 prefix-match hit, remaining 69 prompt tokens to eval
llama_perf_c